In [1]:
import pandas as pd
from wmfdata import hive, charting
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import pct_str

You are using wmfdata 0.1.0 (latest).

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
#clear cache, restart kernel, then clear cache again then run the pip install below
rm -rf ~/.cache/pip/http/*

## Read and clean data

In [7]:
tp_filepath = "../../data/raw/editors/2019/tiger2_participants_report/total_participants.csv"
sr_filepath = "../../data/raw/editors/2019/tiger2_participants_report/support_recipients.csv"

#participants.to_csv(filepath, index=False, header=False)
total_participants = pd.read_csv(tp_filepath)
support_recipients = pd.read_csv(sr_filepath)

In [8]:
#rename column for later merging
total_participants = total_participants.rename(columns={'Which incentive?':'Incentive type',
                                                       'Recieved incentive?': 'Received incentive?'
                                                       })

support_recipients = support_recipients.rename(columns={'Recieved incentive?': 'Received incentive?'
                                                       })

In [9]:
#identify editors not in editor list
e_p = list(total_participants['Username'])
e_r = list(support_recipients['Username'])

support_editors_missing_from_participants = list(set(e_r)-set(e_p))
srta = support_recipients.loc[support_recipients['Username'].isin(support_editors_missing_from_participants)]

In [10]:
srta['glow_editor_signup'] = 'no'
total_participants['glow_editor_signup'] = 'yes'

/home/iflorez/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
participants_recipients_all = pd.concat([total_participants, srta])

In [12]:
#identify dupes 
#articles_w_QID_duplicated = df_w_ids_no_nulls[df_w_ids_no_nulls.duplicated(subset=['Language', 'Articles', 'wiki_db', 'QID'], keep=False)]

#keep only first instance of dupes of full row duplicates
dupes = participants_recipients_all[participants_recipients_all.duplicated(subset=['Username', 'Received incentive?', 'Incentive type'], keep=False)]

In [13]:
dupes = dupes.reset_index().drop('index', axis=1).sort_values('Username')

In [14]:
participants_recipients_all.fillna(0, inplace=True)

In [15]:
participants_recipients_all['glow_editor_signup'] = participants_recipients_all['glow_editor_signup'].replace(to_replace=['No', 'Yes'], value=[0, 1])
participants_recipients_all['glow_editor_signup'] = participants_recipients_all['glow_editor_signup'].replace(to_replace=['no', 'yes'], value=[0, 1])

In [16]:
participants_recipients_all['Received incentive?'] = participants_recipients_all['Received incentive?'].replace(to_replace=['No', 'Yes'], value=[0, 1])
#participants_recipients_all['Received incentive?'] = participants_recipients_all['Received incentive?'].replace(to_replace=['no', 'yes'], value=[0, 1])

In [17]:
participants_recipients_all['Received incentive?'] = participants_recipients_all['Received incentive?'].astype('bool')
participants_recipients_all['glow_editor_signup'] = participants_recipients_all['glow_editor_signup'].astype('bool')

In [ ]:
#TODO Incentive type > categorical (can do ordering with pandas)..but Hive may not allow for categorical. parquet maybe

## Preparing to create table and load into hive

In [18]:
participants_recipients_all.to_csv("../../data/processed/participants_list_received_clean/2019/participants_recipients_all.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
!hive -e LOAD DATA LOCAL INPATH "../../data/processed/participants_list_received_clean/2019/participants_recipients_all.csv" 
INTO TABLE florez.glow_tiger_participants;

# Getting a list of participants into Hive

In [4]:
participants_recipients_all = "../../data/processed/participants_list_received_clean/2019/participants_recipients_all.csv"
hive.load_csv(
    participants_recipients_all,
    field_spec="user_name string, incentive_recipient boolean, incentive_type string, glow_editor_signup boolean",
    db_name="florez",
    table_name="glow_tiger_participants",
    sep=','
)

log4j:WARN No such property [maxBackupIndex] in org.apache.log4j.DailyRollingFileAppender.

Logging initialized using configuration in file:/etc/hive/conf.analytics-hadoop/hive-log4j.properties
Loading data to table florez.glow_tiger_participants
OK
Time taken: 1.657 seconds

